In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import ChatPromptTemplate

import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain_community.chat_models import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough



In [3]:
import os
from dotenv import load_dotenv
load_dotenv()


os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
local_llm = "mistral:latest"
print("loaded env variables")

loaded env variables


In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma


# Load
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=GPT4AllEmbeddings(),
)
retriever = vectorstore.as_retriever()

In [5]:
from typing import Annotated, Dict, TypedDict

from langchain_core.messages import BaseMessage


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        keys: A dictionary where each key is a string.
    """

    keys: Dict[str, any]

In [48]:


### Nodes ###


def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    print("state >>> ", state)
    state_dict = state["keys"]
    question = state_dict["question"]
    # local = state_dict["local"]
    documents = retriever.get_relevant_documents(question)
    return {"keys": {"documents": documents, "question": question}}


def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains generation
    """
    print("---GENERATE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]
    # local = state_dict["local"]

    # Prompt
    prompt = hub.pull("rlm/rag-prompt")

    # LLM
    # if local == "Yes":
    llm = ChatOllama(model="mistral:7b-instruct", temperature=0)
    # else:
    #     # llm = ChatMistralAI(
    #     #     model="mistral-medium", temperature=0, mistral_api_key=mistral_api_key
    #     # )

    #     pass

    # Post-processing
    # def format_docs(docs):
    #     return "\n\n".join(doc.page_content for doc in docs)

    # Chain
    rag_chain = prompt | llm | StrOutputParser()

    # Run
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {
        "keys": {"documents": documents, "question": question, "generation": generation}
    }


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with relevant documents
    """

    print("---CHECK RELEVANCE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]
    # local = state_dict["local"]

    # LLM
    # if local == "Yes":
    llm = ChatOllama(model=local_llm, format="json", temperature=0)
    # else:
    #     # llm = ChatMistralAI(
    #     #     mistral_api_key=mistral_api_key, temperature=0, model="mistral-medium"
    #     # )
    #     pass

    prompt = PromptTemplate(
        template="""You are a grader assessing relevance of a retrieved document to a user question. \n
        Here is the retrieved document: \n\n {context} \n\n
        Here is the user question: {question} \n
        If the document contains keywords related to the user question, grade it as relevant. \n
        It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
        Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
        Provide the binary score as a JSON with a single key 'score' and no premable or explaination.""",
        input_variables=["question","context"],
    )

    chain = prompt | llm | JsonOutputParser()

    # Score
    filtered_docs = []
    search = "No"  # Default do not opt for web search to supplement retrieval
    for d in documents:
        score = chain.invoke(
            {
                "question": question,
                "context": d.page_content,
            }
        )
        grade = score["score"]
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            search = "Yes"  # Perform web search
            continue

        

    return {
        "keys": {
            "documents": filtered_docs,
            "question": question,
            # "local": local,
            "run_web_search": search,
        }
    }

def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]
    # local = state_dict["local"]

    # Create a prompt template with format instructions and the query
    prompt = PromptTemplate(
        template="""You are generating questions that is well optimized for retrieval. \n
        Look at the input and try to reason about the underlying sematic intent / meaning. \n
        Here is the initial question:
        \n ------- \n
        {question}
        \n ------- \n
        Formulate an improved question: """,
        input_variables=["question"],
    )

    # Grader
    # LLM
    # if local == "Yes":
    llm = ChatOllama(model="mistral:7b-instruct", temperature=0)
    # else:
    #     # llm = ChatMistralAI(
    #     #     mistral_api_key=mistral_api_key, temperature=0, model="mistral-medium"
    #     # )
    #     pass

    # Prompt
    chain = prompt | llm | StrOutputParser()
    better_question = chain.invoke({"question": question})

    return {
        "keys": {"documents": documents, "question": better_question}
    }


def web_search(state):
    """
    Web search based on the re-phrased question using Tavily API.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Web results appended to documents.
    """

    print("---WEB SEARCH---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]
    # local = state_dict["local"]

    tool = TavilySearchResults()
    docs = tool.invoke({"query": question})
    print(docs)
    filtered_contents = [d["content"] for d in docs if d["content"] is not None]
    web_results = "\n".join(filtered_contents)
    web_results = Document(page_content=web_results)
    documents.append(web_results)

    return {"keys": {"documents": documents, "question": question}}


### Edges


def decide_to_generate(state):
    """
    Determines whether to generate an answer or re-generate a question for web search.

    Args:
        state (dict): The current state of the agent, including all keys.

    Returns:
        str: Next node to call
    """

    print("---DECIDE TO GENERATE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    filtered_documents = state_dict["documents"]
    search = state_dict["run_web_search"]

    if search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: TRANSFORM QUERY and RUN WEB SEARCH---")
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"

In [49]:
import pprint

from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("generate", generate)  # generatae
workflow.add_node("transform_query", transform_query) 
workflow.add_node("web_search", web_search)  # web search

# # Build graph
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "web_search")
workflow.add_edge("web_search", "generate")
workflow.add_edge("generate", END)

# Compile
app = workflow.compile()

In [50]:
app.invoke({"keys":{"question":"who is naruto"}})

---RETRIEVE---
state >>>  {'keys': {'question': 'who is naruto'}}
---CHECK RELEVANCE---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---DECIDE TO GENERATE---
---DECISION: TRANSFORM QUERY and RUN WEB SEARCH---
---TRANSFORM QUERY---
---WEB SEARCH---
[{'url': 'https://anime.stackexchange.com/questions/7314/why-is-narutos-name-naruto', 'content': "5. As you may know, Jiraiya was an author, as well as the teacher of Minato, Naruto's father. His first book, The Tale of the Utterly Gutsy Shinobi featured a protagonist who held the ideals of Jiraiya and was called - Naruto (possibly named so after the food). Minato liked the book very much and decided to name his son after this character."}, {'url': 'https://naruto-fandom.fandom.com/wiki/Naruto_Uzumaki', 'content': 'Naruto Uzumaki (うずまきナルト, Uzumaki Naruto) is the title character and main protagonist of the series: Naruto. He is a genin-level shinobi 

{'keys': {'documents': [Document(page_content="5. As you may know, Jiraiya was an author, as well as the teacher of Minato, Naruto's father. His first book, The Tale of the Utterly Gutsy Shinobi featured a protagonist who held the ideals of Jiraiya and was called - Naruto (possibly named so after the food). Minato liked the book very much and decided to name his son after this character.\nNaruto Uzumaki (うずまきナルト, Uzumaki Naruto) is the title character and main protagonist of the series: Naruto. He is a genin-level shinobi from Konohagakure, and a member ofTeam Kakashi. A maternal descendant of the Uzumaki clan, Naruto is the third and current jinchūriki of Kurama — the Nine-Tails. Despite once being ostracised by the majority of the village, he is now regarded ...\nNaruto Uzumaki (Japanese: うずまき ナルト, Hepburn: Uzumaki Naruto) (/ ˈ n ɑː r u t oʊ /) is the titular protagonist of the manga Naruto, created by Masashi Kishimoto.He is a ninja from the fictional Hidden Leaf Village (Japanese: 